In [66]:
import pandas as pd

In [67]:
orders = pd.read_csv('data/orders.csv', index_col='payment_id')
orders.head(10)

,user_id,created_at,payment_sum
payment_id,,,
2,2,2019-12-09 16:57:19.000,42.26
4,4,2019-12-10 03:49:35.000,18.25
6,6,2019-12-12 00:19:08.000,10.52
7,7,2019-12-13 15:15:30.000,14.02
8,8,2019-12-14 00:02:06.000,10.78
10,10,2019-12-17 04:57:55.000,27.68
12,12,2019-12-18 23:50:11.000,12.71
15,15,2019-12-20 18:28:57.000,19.06
16,16,2019-12-21 19:10:52.000,17.82


In [68]:
# сколько нужно денег выделить на рекламу в след месяце,
# чтобы увеличить выручку в 2 раза

# сейчас выручка = Х =21К
# нам в след месяце нужно получить 2Х = 42К
# итоговая выручка 
    # выручка от новых клиентов = 27.3к
    # + ыручка от текущих клиентов = 14.7к
# какую долю выручку можно ожидать от текущих клиентов? (фокус сюда сейчас)
# остаток - то, что нужно «докупить» маркетингом

In [69]:
# 1. Построим календарный когортный отчёт 
# - чтобы разбить выручку на когорты (новые-старые)

# 2. Считаем долю выручку от новых клиентов 
# 3. Используем это значение для прогноза

In [70]:
# Алгоритм
# 0. Формилируем задачу
# 1. Определяем когорту (событие и временной промежуток) - первая покупка по месяцам
# 2. Выделяем целевую метрику (выручка)
# 3. Выбираем подходящий формат отчёта (мы будем строить календарные отчёты)

In [71]:
# загрузили данные по выручке
orders = pd.read_csv('data/orders.csv', index_col='payment_id')
orders['created_at'] = pd.to_datetime(orders['created_at'])

# находим первый заказ каждого пользователя через группировку по минимальной дате
orders = (
    orders
    .groupby('user_id')
    .agg(first_payment_at = ('created_at', 'min'))
    .merge(orders, how='inner', left_index=True, right_on='user_id')
) 
orders['payment_month'] = orders['created_at'].dt.to_period('M')
orders['first_payment_month'] = orders['first_payment_at'].dt.to_period('M')


In [72]:
# делаем когортный отчёт через groupby

(
    orders
    .groupby(['first_payment_month', 'payment_month'])
    .agg(revenue = ('payment_sum', 'sum'))
).tail(15)

revenue
first_payment_month payment_month         
2022-07             2022-07        9460.27
                    2022-08        3278.65
                    2022-09        2072.70
                    2022-10        2167.43
                    2022-11        1704.33
2022-08             2022-08        7477.58
                    2022-09        2586.00
                    2022-10        2086.11
                    2022-11        2049.47
2022-09             2022-09        6778.57
                    2022-10        2385.74
                    2022-11        1534.92
2022-10             2022-10        6904.14
                    2022-11        1846.74
2022-11             2022-11        5550.26

In [79]:
#через пивоты
cohorts = (
    orders
    .pivot_table(
        index='first_payment_month', 
        columns='payment_month',
        values='payment_sum',
        aggfunc='sum'
    )
)

cohorts.head(10)

payment_month,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,...,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11
first_payment_month,,,,,,,,,,,,,,,,,,,,,
2019-12,479.2,101.69,31.55,121.12,151.71,201.28,20.91,43.02,67.02,173.82,...,22.84,69.72,NaN,48.15,NaN,NaN,NaN,NaN,48.96,NaN
2020-01,NaN,3003.31,654.82,248.68,569.30,541.44,434.82,462.53,588.11,412.81,...,167.77,21.69,NaN,176.30,55.18,43.06,11.65,75.14,98.37,31.57
2020-02,NaN,NaN,3272.84,462.27,976.15,465.62,594.64,636.79,357.57,458.33,...,81.78,65.37,38.89,75.35,30.39,28.54,24.88,10.15,22.41,88.75
2020-03,NaN,NaN,NaN,2998.08,743.17,618.11,907.15,1122.27,682.06,518.02,...,12.25,145.61,15.01,190.15,272.13,153.15,72.13,71.04,102.45,14.96
2020-04,NaN,NaN,NaN,NaN,2738.40,792.35,976.39,642.97,582.39,251.99,...,NaN,73.80,55.92,75.14,166.15,128.81,153.18,65.13,169.56,148.63
2020-05,NaN,NaN,NaN,NaN,NaN,3456.27,913.80,1019.39,856.90,720.52,...,162.50,266.28,82.19,155.36,85.02,201.31,47.78,83.61,53.32,26.99
2020-06,NaN,NaN,NaN,NaN,NaN,NaN,3692.28,1108.52,964.74,655.60,...,138.18,153.34,103.62,97.48,19.39,120.43,64.25,13.07,31.27,NaN
2020-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5362.63,1638.81,1299.98,...,223.50,165.73,248.01,129.93,268.36,269.99,260.42,82.09,76.63,72.06
2020-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4969.68,1320.12,...,56.75,236.30,102.36,166.70,136.30,507.61,130.83,139.40,45.91,NaN


[np.float64(479.2),
 np.float64(3003.31),
 np.float64(3272.84),
 np.float64(2998.08),
 np.float64(2738.4)]

In [85]:
[cohorts.at[m,m] for m in cohorts.columns][0:5] 
# вытаскиваем значения первых оплат
# [np.float64(479.2),
# np.float64(3003.31),

cohorts.sum(axis='rows').head(5)
#cчитаем суммы по месяцам
# payment_month
# 2019-12     479.20
# 2020-01    3105.00

[cohorts.at[m,m] for m in cohorts.columns] / cohorts.sum(axis='rows')
# 30% выручки приходит от новых пользвоателей. выше мы сумму от новых пользователей 
# делим на общую сумму за месяц и получаем отношение
# 70% выручки приходит от старых. 

payment_month
2019-12    1.000000
2020-01    0.967250
2020-02    0.826640
2020-03    0.782758
2020-04    0.528778
2020-05    0.568927
2020-06    0.489693
2020-07    0.515731
2020-08    0.464140
2020-09    0.414599
2020-10    0.391675
2020-11    0.342659
2020-12    0.336693
2021-01    0.428515
2021-02    0.257788
2021-03    0.299492
2021-04    0.345021
2021-05    0.381927
2021-06    0.379585
2021-07    0.399384
2021-08    0.322369
2021-09    0.321196
2021-10    0.356536
2021-11    0.428804
2021-12    0.316025
2022-01    0.346994
2022-02    0.310629
2022-03    0.325305
2022-04    0.323955
2022-05    0.383911
2022-06    0.329473
2022-07    0.313939
2022-08    0.287265
2022-09    0.281811
2022-10    0.282895
2022-11    0.253363
Freq: M, dtype: float64

payment_month
2019-12    1.000000
2020-01    0.967250
2020-02    0.826640
2020-03    0.782758
2020-04    0.528778
2020-05    0.568927
2020-06    0.489693
2020-07    0.515731
2020-08    0.464140
2020-09    0.414599
2020-10    0.391675
2020-11    0.342659
2020-12    0.336693
2021-01    0.428515
2021-02    0.257788
2021-03    0.299492
2021-04    0.345021
2021-05    0.381927
2021-06    0.379585
2021-07    0.399384
2021-08    0.322369
2021-09    0.321196
2021-10    0.356536
2021-11    0.428804
2021-12    0.316025
2022-01    0.346994
2022-02    0.310629
2022-03    0.325305
2022-04    0.323955
2022-05    0.383911
2022-06    0.329473
2022-07    0.313939
2022-08    0.287265
2022-09    0.281811
2022-10    0.282895
2022-11    0.253363
Freq: M, dtype: float64